# Preparations

## Numpy and plotting library

In [ ]:
import numpy as np
from math import tan, pi
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
from matplotlib import cm 

## Data loading

### Tomography data and associated arrays

In [ ]:
# load tomography and iasp model
data_korean = np.loadtxt("Koreavel.xyz", skiprows=1)

# Create a mask array for filtering out data for < 40 km depth
depths = data_korean[:,2]
d = depths >= 40.0

# arrays for dVp, dVs and Vs.
dVp = data_korean[d,3] - data_korean[d,4] # delta Vp 
dVs = data_korean[d,5] - data_korean[d,6] # delta Vs
S_data = data_korean[d,6]

# Coordinate arrays for later purposes
longitude = data_korean[d,0]
latitude = data_korean[d,1]
depth    = data_korean[d,2]

# Saturate "spike" values of dVs with 0.201 km/s 
# to prevent conversion to unreasonably low or high temperature.
dVs[ np.abs(dVs) > 0.201 ] = 0.201
"""
for i in range(len(depth)):
    if (S_data_anonmal[i] > 0.201) :
        S_data_anonmal[i] =0.201
    elif (S_data_anonmal[i] < -0.201) :
        S_data_anonmal[i] =-0.201    
"""
# Checking minmax of dVs
print(max(dVs), min(dVs))
# Checking the sizes of the coordinate array
print (len(np.unique(depth)), len(np.unique(latitude)), len(np.unique(longitude)))

### Reference geotherm and adiabat 
**[EC]** What is the assumed composition for the Burnman calculation? Ol:Opx:Gt = 0.82:0.144:0.036 below?

In [ ]:
# Load adiabat computed from Burnman
adiabat = np.loadtxt('adiabat_pt.txt')

# Load the reference geotherm (IASP?)
T_ref = np.loadtxt('reference_temp_c.txt', skiprows=1)

### Reference elastic and anelastic properties of minerals

In [ ]:
## From Table A.1 in (Cammarano et al., PEPI, 2003)
data = np.array([('ol', 0.82, 4.2, 1.4, -0.017e9, -0.014e9, 129e9, 81e9, 3222, 0.201e-4, 0.139e-7, 0.1627e-2, 
                   1.658e2, 0.2332e-2, -0.3971e7) , 
                 ('opx', 0.144, 7, 1.6, -0.027e9, -0.012e9, 109e9, 75e9, 3215, 0.3871e-4, 0.0446e-7, 0.03435e-2, 
                   1.855e2, 0.233e-2, -0.6326e7) , 
                 ('gt', 0.036, 4.4, 1.4, -0.019e9, -0.01e9, 171e9, 92e9, 3565, 0.0991e-4, 0.1165e-7, 1.0624e-2, 
                  1.44e2, 0.188e-2, -0.135e7)], 
        dtype=[('name', 'U10'), ('conc', 'f4'), ('KPDer', 'f4'), ('MuPDer', 'f4'), 
        ('KTDer', 'f4'), ('MuTDer', 'f4'), ('K', 'f4'), ('Mu', 'f4'), ('rho', 'f4'), 
        ('a0', 'f4'), ('a1', 'f4'), ('a2', 'f4'), ('cp0', 'f4'), ('cp1', 'f4'), ('cp2', 'f4')])

## Anelasticity Parameter 
## There is original reference in Goes and Govers, 2000, Shallow mantle temperatures under Europe from P and S
## wave tomography, JGR, 97
A = 1.48e-1 ; H = 500e3; V = 20e-6; a = 0.15 ; d=1e-3
R = 8.314
grad_crust = -0.17; ## Yan et al., 2016, Stress development in heterogenetic lithosphere. Techtonophysics, 671, pp 56-62.
grad_crust2 = -0.25; ## Yan et al., 2016, Stress development in heterogenetic lithosphere. Techtonophysics, 671, pp 56-62.

# Procedure

We closely follow the method described in Appendix A in [Cammarano et al., PEPI, 2003] for computing density, elastic moduli and their partial derivatives with respect to temperature and pressure.

Their main workflow is
1. Set a target $(P,T)$ where the reference mineral properties are extrapolated to.
2. Compute an adiabat that goes through $(P,T)$ and acquire a surface potential temperature $T_{pot}$.
3. Extrapolate properties from the reference temperature ($T_{0}$, room temp?) to $T_{pot}$.
4. Extrapolate properties at $T_{pot}$ from the reference pressure ($P_{0}$, atmospheric pressure?) to $P$ **along** the adiabat. Note that, since the adiabat is followed, temperature at the end would correspond to the target value $T$, not $T_{pot}$.

Saxena et al. (in prep) simplies the workflow skipping the steps 1 and 2. **The rationale is [TBA].**



## Density extrapolation 
### From ($T_{0}$, $P_{0}$) to ($T_{pot}$, $P_{0}$)
Follwoing eq. (6) in (Duffy and Anderson, JGR, 1989) and the 2nd eq. in Appendix A of (Cammarano et al., PEPI, 2003), we assume that for a fixed pressure $P_{0}$, density at temperature $T_{pot}$ is extrapolated from a reference value at $T_{0}$ as follows **[EC: Physical or mathematical origin of this particular form unknown yet]**:
\begin{equation}
\rho(T_{pot}, P_0) = \rho(T_{0}, P_0)\exp\left( -\int_{T_0}^{T_{pot}} \alpha(T^{\prime})\ dT^{\prime} \right), \qquad \text{(eq.1)}
\end{equation}
where $\alpha$ is thermal expansion coefficient. 

We adopt the following polynomial form of $\alpha$ [Saxena and Shen, 1992, JGR, 97, pp 19813-19825]: 
\begin{equation}
  \alpha(T)=\alpha_0 + \alpha_1 T + \alpha_2 T^{-1} + \alpha_3 T^{-2} + \ldots.
\end{equation}

Plugging the polynomial form into eq. 1, we get
<!--\begin{equation}
\rho(T, P_0) = \rho(T_0, P_0) \exp \left(-(\alpha_0(T-T_0) + \frac{1}{2}\alpha_1(T-T_0)^2 + \alpha_2ln(T-T_0) + ...) \right). \qquad \text{(eq.2)}
\end{equation}-->
\begin{equation}
\begin{split}
&\rho(T_{pot}, P_0) = \\
&\rho(T_0, P_0) \exp \left(-\left[\alpha_{0}\,(T_{pot}-T_0) + \frac{1}{2}\alpha_{1}\,(T_{pot}^{2}-T_{0}^{2}) + \alpha_2\,(\ln(T_{pot})-\ln(T_{0})) + \ldots \right] \right). \qquad \text{(eq.2)}
\end{split}
\end{equation}

<!--**[LSH]**: I think the original eq.2 is correct because second term disappear due to (T-T0)^2-(T0-T0)^2 **-->

<!---### Notes from the code block for density function:
- We should check what T0 means (atmophere temperature or surface temperautre)
- In equation 1, T0 means 293 K (i.e. atmospheric temperature) 
- Equation 1 means that density changes in temperature using polynomoal thermal expansion with depth--->

In [ ]:
def density(a0, a1, a2, rho0, Tpot, T0 = 300.0):
    """ 
    rho0: Reference density, rho(T0, p0)
    T0: Reference temperature set to 300.0 K by default 
        but a different value can be given
    For other parameters, refer to eq.2 derived above.
    """
    alpha_integ = a0*(T - T0) + 0.5*a1*(Tpot**2 - T0**2) + a2*(np.log(Tpot) - np.log(T0))
    rho_extrap = rho0 * np.exp(-alpha_integ)
    return rho_extrp
"""
def density(a0, a1, a2, rho0, T):
    T0 = 300.  # surface temperature and T should be potential temperature (1600 K)
    return (rho0 * np.exp (-a0 * (T - T0) - ( a1*(T - T0)**2 )/2 - a2*np.log (T - T0))) 
"""

### From ($T_{pot},P_{0}$) to ($T,P$)
We solve for $\rho(T,P)$ the following equation (see Appendix A in Cammarano et al., PEPI, 2003) relating the Eulerian strain ($\varepsilon$) and density:
\begin{equation}
  \varepsilon = \frac{1}{2} \left[ 1- \left( \frac{\rho(T,P)}{\rho(T_{pot},P_{0})} \right)^{2/3} \right]. \qquad \text{(eq.3)}.
\end{equation}
Also from Appendix A in (Cammarano et al., PEPI, 2003), $\varepsilon$ is related to the target pressure $P$ via
\begin{equation}
  P = -(1 - 2\varepsilon)^{5/2}\, \left( 3K\varepsilon + \frac{1}{2}\,9K(4-K')\,\varepsilon^2 + \ldots \right), \qquad \text{(eq.4)}
\end{equation}
where $K$ and $K'$ are bulk modulus and its pressure derivative.

We define a function, $f(\varepsilon)$ with a truncated version of eq.(4) and the target pressure $P$; and apply the Newton-Raphson method to find a zero root as the desired value of $\varepsilon$:
\begin{equation}
  f(\varepsilon) = P + (1 - 2\varepsilon)^{5/2}\, \left( 3K\varepsilon + \frac{1}{2}\,9K(4-K')\,\varepsilon^2 \right). \qquad \text{(eq.5)}
\end{equation}
The found value of $\varepsilon$ is used in eq.(3) to get $\rho(T,P)$ as
\begin{equation}
  \rho(T, P)=(1-2\varepsilon)^{3/2}\,\rho(T_{pot}, P_0) \qquad \text{(eq.6)}
\end{equation}

In [ ]:
def derivative0(f, x, h, P, K, Kprime):
    """ 
    Computes df/dx using the central difference scheme.
    f: a function to be differentiated
    x: independent variable for f(x)
    h: small internal in x
    """
    return (f(x+h, P, K, Kprime) - f(x-h, P, K, Kprime)) / (2.0*h) # might want to return a small non-zero if ==0

def fepsilon(x, P, K, Kprime):
    """ Implementation of eq. 5 defined above."""
    return P + (1.0-2.0*x)**(2.5) * ( 3.0*K*x + 4.5*K*(4.0-Kprime)*(x**(2.0)) )

def solve_ep(f, x0, h, P, K, Kprime):
    lastX = x0
    error = np.abs(10.0*h)  # "different than lastX so loop starts OK
    while (errorr > h):  # this is how you terminate the loop - note use of abs()
        fval  = fepsilon(lastX, P, K, Kprime) # just for debug... see what happens
        dfdx  = derivative0(fepsilon, lastX, h, P, K, Kprime)
        nextX = lastX - fval / dfdx  # update estimate using N-R
        error = np.abs(nextX - lastX)
        lastX = nextX
    return nextX
"""
def solve_ep (f, x0, h, P, K, Kprime):
    lastX = x0
    nextX = lastX + 10 * h  # "different than lastX so loop starts OK
    while (np.abs(lastX - nextX) > h):  # this is how you terminate the loop - note use of abs()
        newY  = epsilon(nextX,  P, K, Kprime)                     # just for debug... see what happens
        lastX = nextX
        nextX = lastX - newY / derivative0(f, lastX, h, P, K, Kprime)  # update estimate using N-R
    return nextX
"""

## Simplified density extrapolation
Saxena et al. (in prep) choose to assume a single representative potential temperature, 1600 K, rather than computing $T_{pot}$ for each $(T,P)$ in the reference geotherm. This approach is justified above.

In [ ]:
x0 = 1.e-8
pressures = adiabat[:,0]
depth_a =  adiabat[:,1] / 1e3 # tomography is in m, easier for comparison
#T_a = adiabat[:,2]
T_r = T_ref[:,1] # reference temp with depth
strain = np.ones(len(pressures))
rho = np.ones(len(pressures))

# Mineral concentration-weighted average elastic properties
# for the assumed rock composition.
rho_avg = np.dot( data['rho'], data['conc']) # rho_i * c_i
K_avg = np.dot( data['K'], data['conc'])     # K_i * c_i etc.
Muavg = np.dot( data['Mu'], data['conc'])
KPDer_avg = np.dot( data['KPDer'], data['conc'])
print(K_avg, Muavg, KPDer_avg)
#rho_avg = data['rho'][0] * data['conc'][0] + data['rho'][1] * data['conc'][1] + \
#            data['conc'][2] * data['rho'][2]
#K_avg = ( data['conc'][0] * data['K'][0] + data['conc'][1] * data['K'][1] + data['conc'][2]*data['K'][2])
#Muavg = ( data['conc'][0] * data['Mu'][0] + data['conc'][1] * data['Mu'][1] + data['conc'][2]*data['Mu'][2])
#KPDer_avg = ( data['conc'][0] * data['KPDer'][0] + data['conc'][1] * data['KPDer'][1] + \
#          data['conc'][2]*data['KPDer'][2])

def call_density( data, Tpot ):
    num_minerals = len(data['conc'])
    densities = np.zeros( num_minerals )
    for i in range(num_minerals):
        a0 = data['a0'][i]
        a1 = data['a1'][i]
        a2 = data['a2'][i]
        rho0 = data['rho'][i]
        densities[i] = density( a0, a1, a2, rho0, Tpot )
    return densities        

for i in range (len(pressures)):
    # Compute strain
    strain[i] = solve_ep (epsilon, x0, 1.e-15, pressures[i], K_avg, KPDer_avg)
    # Compute densities for mineral components
    densities = call_density( data, 1600.0 )
    rho_pot = np.dot( data['conc'], densities ) # c_i * rho_i
    # eq.(6) in this document.
    rho[i] = rho_pot * ( (1.0 - 2.0*strain[i])**1.5 )
    #rho[i] = (( data['conc'][0] *  density ( data['a0'][0], data['a1'][0], data['a2'][0], data['rho'][0], 1600) ) 
    #+ ( data['conc'][1] *  density ( data['a0'][1], data['a1'][1], data['a2'][1], data['rho'][1], 1600) )
    #+ ( data['conc'][2] *  density ( data['a0'][2], data['a1'][2], data['a2'][2], data['rho'][2], 1600) ) ) \
    #* ( ( 1 - ( 2 * strain[i] ) ) ** (3/2) )
    
    print (depth_a[i], T_r[i], strain[i], rho[i])

<!--$$ \partial M = \left(\left(\frac{\partial K}{\partial T}\right )_P+4/3\left(\frac{\partial G}{\partial T}\right )_P \right)(T-T_0) \dots (eq.6) $$-->
\begin{equation}
\partial M = \left(\left(\frac{\partial K}{\partial T}\right )_P+4/3\left(\frac{\partial G}{\partial T}\right )_P \right)\partial T \approx  \left(\left(\frac{\partial K}{\partial T}\right )_P+4/3\left(\frac{\partial G}{\partial T}\right )_P \right)\Delta T \qquad \text{(eq.5)}
\end{equation}

<!--$$ \partial G = \left(\frac{\partial G}{\partial T}\right )_P(T-T_0) \dots (eq.8)$$-->
\begin{equation}
\partial G = \left(\frac{\partial G}{\partial T}\right )_P\partial T \approx \left(\frac{\partial G}{\partial T}\right )_P\Delta T \qquad \text{(eq.6)}
\end{equation}

\begin{equation}
\partial M'(T) = \left(\left(\frac{\partial K}{\partial P}\right )_T+4/3\left(\frac{\partial G}{\partial P}\right )_T \right)\exp(\int\limits_{T_0}^T \alpha(T')\ dT')\alpha(T')\partial T \approx \qquad \left(\left(\frac{\partial K}{\partial P}\right )_T+4/3\left(\frac{\partial G}{\partial P}\right )_T \right)\exp(\int\limits_{T_0}^T \alpha(T')\ dT')\alpha(T')\Delta T \qquad \text{(eq.7)}
\end{equation}

In [ ]:
# moduli plus temperature (potenttial) corrrection term 
def pmodT ( To, data, x, comp , ep ):
    T0=300
    Tp=1600
    return  ( data['K'][comp] + (4./3)*data['Mu'][comp] + 
             data['KTDer'][comp] * (Tp - To) + (4./3) * data['MuTDer'][comp] * (Tp - To) )

def smodT ( To, data, x, comp , ep ):
    T0=300
    Tp=1600
    return  ( data['Mu'][comp] + data['MuTDer'][comp] * (Tp - To) )

In [ ]:
# temperature corrrection term for the moduli
def moduli ( To, data, x, comp):
    return  ( data['KTDer'][comp] * (x - To) + (4./3) * data['MuTDer'][comp] * (x - To) )

def s_moduli ( To, data, x, comp):
    return  ( data['MuTDer'][comp] * (x - To) )

In [ ]:
def moduli_der0 ( To, data, x, comp , ep ):
    T0=300
    return ( data['KPDer'][comp] + (4/3) * data['MuPDer'][comp] ) *   \
           ( np.exp ( data['a0'][comp] * (x - T0) + ( data['a1'][comp] / 2 ) * (x - T0)**2) )

def smoduli_der0 ( To, data, x, comp , ep ):
    T0=300
    return ( data['MuPDer'][comp] ) *   \
           ( np.exp ( data['a0'][comp] * (x - T0) + ( data['a1'][comp] / 2 ) * (x - T0)**2) )

In [ ]:
def moduli_der (To, data, x, comp):
    T0=300
    return ( data['KPDer'][comp] + (4/3) * data['MuPDer'][comp] ) *   \
           ( ( np.exp ( data['a0'][comp] * (x - T0) + ( data['a1'][comp] / 2 ) * (x - T0)**2) ) \
            * ( data['a0'][comp] + data['a1'][comp] * (x - T0) )  * ( x - To ) ) 

def smoduli_der (To, data, x, comp):
    T0=300
    return ( data['MuPDer'][comp] ) *   \
           ( ( np.exp ( data['a0'][comp] * (x - T0) + ( data['a1'][comp] / 2 ) * (x - T0)**2) ) \
            * ( data['a0'][comp] + data['a1'][comp] * (x - T0) )  * ( x - To ) ) 

### By definition of eq. 1, T0 is surface temp.
### However, eq. 7 is resulant of derivatives to temperaure difference 
### So, To means reference temp and X means perturbation temp

\begin{equation}
\partial M = (1-2\epsilon)^{5/2}\left\{\frac{\partial M}{\partial T}\partial T+ \epsilon [5\frac{\partial M}{\partial T}\partial T-3\frac{\partial K}{\partial T} \left ( \frac{\partial K}{\partial P} -(4/3) \frac{\partial G}{\partial P}\right )\partial T - 3K\partial M'(T)] \right\} \approx \qquad (1-2\epsilon)^{5/2}\left\{\frac{\partial M}{\partial T}\Delta T+ \epsilon [5\frac{\partial M}{\partial T}\Delta T-3\frac{\partial K}{\partial T} \left ( \frac{\partial K}{\partial P} -(4/3) \frac{\partial G}{\partial P}\right )\Delta T - 3K\partial M'(T)] \right\}\text{(eq.8)}
\end{equation}

\begin{equation}
\partial G = (1-2\epsilon)^{5/2}\left\{\frac{\partial G}{\partial T}\partial T+ \epsilon [5\frac{\partial G}{\partial T}\partial T-3\frac{\partial K}{\partial T}  \frac{\partial G}{\partial P}\partial T - 3K\partial G'(T)] \right\} \approx \qquad (1-2\epsilon)^{5/2}\left\{\frac{\partial G}{\partial T}\Delta T+ \epsilon [5\frac{\partial G}{\partial T}\Delta T-3\frac{\partial K}{\partial T}  \frac{\partial G}{\partial P}\Delta T - 3K\partial G'(T)] \right\} \qquad \text{(eq.9)}
\end{equation}

In [ ]:
# P, T function for K and mu 
def pmodPT ( To, data, x, comp , ep ):
    return ( ( 1 - 2*ep )**(5/2) ) * ( pmodT(data, x, comp) +
            ep * ( 5 * pmodT(data, x, comp)  - 3 * ( pmodT(data, x, comp) - (4/3)*smodT(data, x, comp) ) * \
                  ( moduli_der0 ( To, data, x, comp , ep ) ) ) ) 

def smodPT ( To, data, x, comp , ep ):  
    return ( ( 1 - 2*ep )**(5/2) ) * ( smodT( To, data, x, comp , ep )  +   \
            ep * ( 5 * smodT( To, data, x, comp , ep )  - 3 * ( pmodT( To, data, x, comp , ep ) - (4/3)*smodT( To, data, x, comp , ep ) ) *  \
                 ( smoduli_der0 ( To, data, x, comp , ep ) ) ) )

In [ ]:
# P, T function for K and mu delivative to temperature difference
def p_modulus ( To, data, x, comp , ep ):   
    return ( ( 1 - 2*ep )**(5/2) ) * ( moduli ( To, data, x, comp)   +   \
            ep * ( 5 * moduli ( To, data, x, comp)  - 3 * ( data['KTDer'][comp] ) *  \
          ( x- To ) * ( data['KPDer'][comp]  + (4/3) * data['MuPDer'][comp] ) - 3 * data['K'][comp] *   \
            ( moduli_der (To, data, x, comp ) ) ) ) 

def s_modulus ( To, data, x, comp , ep ):   
    return ( ( 1 - 2*ep )**(5/2) ) * ( s_moduli ( To, data, x, comp)   +   \
            ep * ( 5 * s_moduli ( To, data, x, comp)  - 3 * ( data['KTDer'][comp] ) *  \
          ( x- To ) * (  data['MuPDer'][comp] ) - 3 * data['K'][comp] * (3/4) * \
            ( moduli_der (To, data, x, comp ) ) ) )  

In [ ]:
def density_der(comp, rho, To, data, x, ep):
    T0=300
    Tp=1600
    return - data['rho'][comp] * np.exp (- data['a0'][comp] * (Tp - T0) - \
        ( data['a1'][comp] / 2) * (Tp - T0) ** 2  ) * ( data['a0'][comp] + data['a1'][comp] * (x - 300) ) * \
        ( x - To) * ( 1 - 2*ep )**(3/2.) 

In [ ]:
# inversion for temperature
def double_derivative (f, x, h, delVp, ep, rho, To, data, P, Vso, Vpo):
    return (f(x+h, delVp, ep, rho, To, data, P, Vso, Vpo) + f(x-h, delVp, ep, rho, To, data, P, Vso, Vpo) -
            2 * f(x-h, delVp, ep, rho, To, data, P, Vso, Vpo) ) / (h**2)  

def derivative (f, x, h, delVp, ep, rho, To, data, P, Vso, Vpo):
    return (f(x+h, delVp, ep, rho, To, data, P, Vso, Vpo) - f(x-h, delVp, ep, rho, To, data, P, Vso, Vpo)  ) / (2 * h)

$$\partial V_p / \partial T=  \partial V_{panh} / \partial T +\partial V_{pane} / \partial T \dots (eq.11)$$

$$ where, $$


$$Q_p^{-1} = \left (1-4/3 \left (\frac{V_s}{V_p} \right )^2 \right )Q_k^{-1} + 4/3 \left (\frac{V_s}{V_p} \right )^2 \left ( Aw^a\exp \left (\frac {a(E+PV)}{RT}  \right ) \right )^{-1} \dots (eq.12) $$

$$ \partial V_p = \frac{1}{\sqrt \rho} \frac{1}{2\sqrt {K+4G/3}}\partial (K+4G/3)-\frac {\sqrt {K+4G/3}}{2 \rho^{3/2}} \partial \rho + {Q_p}^-1\frac{aH}{2RT^2 \tan {\frac {\pi a}{2}}}(T-T_r) \dots (eq.13)$$

$$\partial V_s / \partial T=  \partial V_{sanh} / \partial T +\partial V_{sane} / \partial T \dots (eq.14)$$

$$ where, $$

$$Q_s^{-1} = \left ( Aw^a\exp \left (\frac {a(E+PV)}{RT}  \right ) \right )^{-1} \dots (eq.15) $$

$$ \partial V_s = \frac{1}{\sqrt \rho} \frac{1}{2\sqrt G}\partial G-\frac {\sqrt G}{2 \rho^{3/2}} \partial \rho + {Q_s}^-1\frac{aH}{2RT^2 \tan {\frac {\pi a}{2}}}(T-T_r) \dots (eq.16)$$

In [ ]:
def temperature_p ( x, delVp, ep, rho, To, data, P, Vso, Vpo):
    p_modulus0 = K_avg + (4/3)*Muavg     
    return ( 1/(2 * np.sqrt(rho_avg * p_modulus0) ) ) * ( data['conc'][0] *  p_modulus ( To, data, x, 0 , ep ) +
            data['conc'][1] *  p_modulus ( To, data, x, 1 , ep ) + data['conc'][2] *
            p_modulus ( To, data, x, 2 , ep ) ) - ( ( np.sqrt(p_modulus0)/( 2 * ( (rho_avg) ** 3/2 ) ) ) * \
            ( data['conc'][0] * density_der (0, rho, To, data, x , ep) +  data['conc'][1] * 
             density_der (1, rho, To, data, x, ep) + data['conc'][2] * density_der (2, rho, To, data, x, ep ) ) ) \
             + ( ( a * H * (4/3) * (Vso/Vpo)**2 ) * ( ( A * ( ( 2*np.pi )**a ) * 
             np.exp ( a * ( H + P*V ) / ( R * To ) ) ) ** -1 ) /(2 * R * ( To ** 2) * np.tan ( np.pi * a/2 ) ) ) \
             * (x - To) - delVp

In [ ]:
def temperature_s ( x, delVs, ep, rho, To, data, P, Vso, Vpo):
    data['KPDer'][:] = 0;
    return ( 1/(2 * np.sqrt(rho * (data['conc'][0] * smodPT (To, data, x, 0 , ep) + data['conc'][1] * smodPT (To, data, x, 1 , ep) + data['conc'][2] * smodPT (To, data, x, 2 , ep))) ) ) * (data['conc'][0] * ( s_modulus ( To, data, x, 0 , ep ) +
            data['conc'][1] *  s_modulus ( To, data, x, 1 , ep ) + data['conc'][2] *
            s_modulus ( To, data, x, 2 , ep ) ) ) - ( ( np.sqrt((data['conc'][0] * smodPT (To, data, x, 0 , ep) + data['conc'][1] * smodPT (To, data, x, 1 , ep) + data['conc'][2] * smodPT (To, data, x, 2 , ep)))/( 2 * ( rho ** 3/2 ) ) ) * \
            ( data['conc'][0] * density_der (0, rho, To, data, x , ep) +  data['conc'][1] * 
             density_der (1, rho, To, data, x, ep) + data['conc'][2] * density_der (2, rho, To, data, x, ep ) ) ) \
             + ( ( a * H ) * ( ( A * ( ( 2*np.pi )**a ) * 
             np.exp ( a * ( H + P*V ) / ( R * To ) ) ) ** -1 ) /(2 * R * ( To ** 2) * np.tan ( np.pi * a/2 ) ) ) \
             * (x - To) - delVs
                    
#     s_modulus0 = Muavg  
#     data['KPDer'][:] = 0 ;
#     return ( 1/(2 * np.sqrt(rho * s_modulus0) ) ) * (data['conc'][0] * ( s_modulus ( To, data, x, 0 , ep ) +
#             data['conc'][1] *  s_modulus ( To, data, x, 1 , ep ) + data['conc'][2] *
#             s_modulus ( To, data, x, 2 , ep ) ) ) - ( ( np.sqrt(s_modulus0)/( 2 * ( rho ** 3/2 ) ) ) * \
#             ( data['conc'][0] * density_der (0, rho, To, data, x , ep) +  data['conc'][1] * 
#              density_der (1, rho, To, data, x, ep) + data['conc'][2] * density_der (2, rho, To, data, x, ep ) ) ) \
#              + ( ( a * H ) * ( ( A * ( ( 2*np.pi )**a ) * 
#              np.exp ( a * ( H + P*V ) / ( R * To ) ) ) ** -1 ) /(2 * R * ( To ** 2) * np.tan ( np.pi * a/2 ) ) ) \
#              * (x - To) - delVs

In [ ]:
def solve_T (f, x0, h, delVp, ep, rho, To, data, P, Vso, Vpo):
    lastX = x0
    nextX = lastX +  10 * h  # "different than lastX so loop starts OK
    while (abs(lastX - nextX) > h):  # this is how you terminate the loop - note use of abs()        
        newY  = f (nextX, delVp, ep, rho, To, data, P, Vso, Vpo)          # just for debug... see what happens
#         print (newY)     # print out progress... again just debug
        lastX = nextX
        nextX = lastX -  f (nextX,  delVp, ep, rho, To, data, P, Vso, Vpo) / derivative(f, lastX, h,  delVp, ep, rho, To, data, P, Vso, Vpo)  # update estimate using N-R
    return nextX

In [ ]:
To = np.ones(len(depth)) ; ep_cal = np.ones(len(depth))
rho_cal = np.ones(len(depth)); T = np.ones(len(depth)) 
P = np.ones(len(depth)) 

In [ ]:
# invert for temperatures
x0 = 500;
depth_a =  adiabat[:,1] / 1e3 # tomography is in m, easier for comparison
T_a = adiabat[:,2]
Vp_o  = data_korean[:,5]*1.e3
Vs_o  = data_korean[:,5]*1.e3

for i in range(len(depth)):
    To[i]      = (T_ref[:,1][5.0 >= np.abs(depth[i]-T_ref[:,0])].max()) # reference temperature with depth
    ep_cal[i]  = strain [5.0 >= np.abs(depth[i]-depth_a)].max()
    P[i]       = (pressures[5.0 >= np.abs(depth[i]-depth_a)].max())
    rho_cal[i] = rho [5.0 >= np.abs(depth[i]-depth_a)].max()
    
# equation to get x
    T[i]     = solve_T (temperature_s, x0, 1,  S_data_anonmal[i]*1e3, ep_cal[i], rho_cal[i], To[i], data, P[i], 
                        Vs_o[i], Vp_o[i])

    if np.mod((i+9597), 9597)== 0:
        print(To[i], Vs_o[i], ep_cal[i], rho_cal[i], depth[i])
        
#     for j in range(10): # Non-linear loop 
#         if j==0:
#             T[i]     = solve_T (temperature_s, x0, 1,  S_data_anonmal[i]*1e3, ep_cal[i], rho_cal[i], To[i], data, P[i], Vs_o[i], Vp_o[i])     
#             k_avg=(data['conc'][0]*(p_modulus0(data, To[i], 0 , ep_cal[i])-4/3*s_modulus0(data, To[i], 0 , ep_cal[i]))+
#                    data['conc'][1]*(p_modulus0(data, To[i], 1 , ep_cal[i])-4/3*s_modulus0(data, To[i], 1 , ep_cal[i]))+
#                    data['conc'][2]*(p_modulus0(data, To[i], 2 , ep_cal[i])-4/3*s_modulus0(data, To[i], 2 , ep_cal[i])))
#             KPDer_avg=(data['conc'][0]*(moduli_der0(data, T[i], 0 )-4/3*smoduli_der0(data, T[i], 0))+
#                        data['conc'][1]*(moduli_der0(data, T[i], 1)-4/3*smoduli_der0(data, T[i], 1))+
#                        data['conc'][2]*(moduli_der0(data, T[i], 2)-4/3*smoduli_der0(data, T[i], 2)))  
#             N_strain[i]     = solve_ep (epsilon, ep0, 1.e-15, P[i], K_avg, KPDer_avg) # equation to get strain
#         else:
#             T[i]     = solve_T (temperature_s, T[i], 1,  S_data_anonmal[i]*1e3, N_strain[i], rho_cal[i], To[i], data, P[i], Vs_o[i], Vp_o[i])     
#             k_avg=(data['conc'][0]*(p_modulus0(data, T[i], 0 , N_strain[i])-4/3*s_modulus0(data, T[i], 0 , N_strain[i]))+
#                    data['conc'][1]*(p_modulus0(data, T[i], 1 , N_strain[i])-4/3*s_modulus0(data, T[i], 1 , N_strain[i]))+
#                    data['conc'][2]*(p_modulus0(data, T[i], 2 , N_strain[i])-4/3*s_modulus0(data, T[i], 2 , N_strain[i])))
#             KPDer_avg=(data['conc'][0]*(moduli_der0(data, T[i], 0 )-4/3*smoduli_der0(data, T[i], 0))+
#                        data['conc'][1]*(moduli_der0(data, T[i], 1)-4/3*smoduli_der0(data, T[i], 1))+
#                        data['conc'][2]*(moduli_der0(data, T[i], 2)-4/3*smoduli_der0(data, T[i], 2)))            
#             lastN = N_strain[i]
#             N_strain[i]     = solve_ep (epsilon, ep0, 1.e-15, P[i], K_avg, KPDer_avg)

In [ ]:
T_results = np.loadtxt('initial_temperature_S_tomography.txt', skiprows=4, usecols=3)

In [ ]:
d_crust = (deep < 20.0) & (deep > 0.0)
lat_c = data_korean[d_crust, 1]
long_c = data_korean[d_crust, 0]
d_c = data_korean[d_crust, 2]
T_crust = ((data_korean[d_crust,5] - data_korean[d_crust,6]) * grad_crust)*1000 + 484.13

d_crust2 = (deep < 40.0) & (deep > 20.0)
lat_c2 = data_korean[d_crust2, 1]
long_c2 = data_korean[d_crust2, 0]
d_c2 = data_korean[d_crust2, 2]
T_crust2 = ((data_korean[d_crust2,5] - data_korean[d_crust2,6]) * grad_crust2)*1000 + 588.05

# print(max(T_crust), min(T_crust), max(T_crust2), min(T_crust2))

In [ ]:
d_q = np.unique(depth)
d_cal   = np.reshape(depth,(len(d_q), 101, 95))
T_cal   = np.reshape(T, (len(d_q), 101, 95))
dVs_cal = np.reshape(S_data_anonmal , (len(d_q), 101, 95))
# Vp_cal = np.reshape(Vp, (len(d_q), npts, npts))

lat_cal = np.reshape(latitude,(len(d_q), 101, 95))
long_cal= np.reshape(longitude,(len(d_q), 101, 95))
n = 15
(fig, (ax1, ax2)) = plt.subplots(1, 2, figsize=(20,6))
plt.rcParams.update({'font.size': 14})
CB = ax1.pcolormesh(long_cal[n,:, :], lat_cal[n, :, :], T_cal[n ,:, :], cmap=cm.hot, shading='gouraud')
CB1 = ax2.pcolormesh(long_cal[n,:, :], lat_cal[n, :, :], dVs_cal[n ,:, :], cmap=cm.hot, shading='gouraud')
cbar = plt.colorbar(CB, shrink=0.6, ax=ax1)
cbar2 = plt.colorbar(CB1, shrink=0.6, ax=ax2)

CB.axes.set_xlim([122, 135])
CB.axes.set_ylim([31, 41])
CB1.axes.set_xlim([122, 135])
CB1.axes.set_ylim([31, 41])
plt.show()
print(np.unique(depth)[n])
print(np.unique(depth))

In [ ]:
print (d_q[14])

In [ ]:
# addting two buffer layer for crust (18.8 km and 38.8 km)

T_c = np.concatenate([ T, T_crust2, T_crust]);
lat = np.concatenate([latitude, lat_c2, lat_c])
longt = np.concatenate([longitude, long_c2, long_c])
depth_c = np.concatenate([depth, d_c2, d_c])
# T_c = np.concatenate([ T]);

# T_c = np.concatenate([ T, T_crust]);
# # T_a = np.concatenate([ AT, T_crust]);
# lat = np.concatenate([latitude, lat_c])
# longt = np.concatenate([longitude, long_c])
# depth_c = np.concatenate([depth,d_c])
# lat = np.concatenate([latitude])
# longt = np.concatenate([longitude])
# depth_c = np.concatenate([depth])

print (np.shape(lat ), np.shape(depth_c), np.shape(T_c))

r = (6370 - depth_c)*1e3
long_f = longt*np.pi/180
lat_f = (90 - lat)*np.pi/180
ind = np.lexsort((r, long_f, lat_f))
np.savetxt('initial_temperature_S_tomography.txt', \
            np.stack((r[ind], long_f[ind], lat_f[ind], T_c[ind]), axis=-1))

In [ ]:
min(T_c)
# print([S_data_anonmal == max(S_data_anonmal)])

In [ ]:
print(long_f[ind])
